In [2]:
import pandas as pd

movies = pd.read_csv("movies_cleaned.csv")

In [13]:
# movies["genre"].value_counts().reset_index()
movies['genre'].str.split(', ').explode().value_counts().reset_index()


,genre,count
0,Drama,11823
1,Comedy,6610
2,Thriller,6007
3,Horror,4795
4,Action,4388
5,Romance,4114
6,Crime,3568
7,Adventure,2580
8,Science Fiction,2508
9,Mystery,2340


In [12]:
# num of genres with 50 or more movies
movies["genre"].str.split(', ').explode().value_counts().reset_index().query("count > 1000")


,genre,count
0,Drama,11823
1,Comedy,6610
2,Thriller,6007
3,Horror,4795
4,Action,4388
5,Romance,4114
6,Crime,3568
7,Adventure,2580
8,Science Fiction,2508
9,Mystery,2340


In [17]:
genre_map = {
    "Drama": "Emotional",
    "Romance": "Emotional",
    "War": "Emotional",
    "Comedy": "Feel-good",
    "Family": "Feel-good",
    "Animation": "Feel-good",
    "Thriller": "Suspenseful",
    "Horror": "Suspenseful",
    "Crime": "Suspenseful",
    "Mystery": "Suspenseful",
    "Action": "Escapist",
    "Adventure": "Escapist",
    "Science Fiction": "Escapist",
    "Fantasy": "Escapist",
    "History": "Educational",
    "Documentary": "Educational",
    "Music": "Artistic",
    "TV Movie": "Classic",
    "Western": "Classic"
}

# movies["simple_genres"] = movies["genre"].map(genre_map)
def map_genres(genre_str):
    genres = genre_str.split(", ")
    mapped = [genre_map[g] for g in genres if g in genre_map]
    return list(set(mapped))  # Optional: remove duplicates

movies["simple_genres"] = movies["genre"].apply(map_genres)

In [18]:
movies

,id,name,date,tagline,description,minute,rating,theme,genre,tagged_description,simple_genres
0,1000001,Barbie,2023.0,She's everything. He's just Ken.,Barbie and Ken are having the time of their li...,114.0,3.86,"Humanity and the world around us, Crude humor ...","Comedy, Adventure",1000001 Barbie and Ken are having the time of ...,"[Escapist, Feel-good]"
1,1000002,Parasite,2019.0,Act like you own the place.,"All unemployed, Ki-taek's family takes peculia...",133.0,4.56,"Humanity and the world around us, Intense viol...","Comedy, Thriller, Drama","1000002 All unemployed, Ki-taek's family takes...","[Feel-good, Suspenseful, Emotional]"
2,1000003,Everything Everywhere All at Once,2022.0,The universe is so much bigger than you realize.,An aging Chinese immigrant is swept up in an i...,140.0,4.30,"Humanity and the world around us, Moving relat...","Science Fiction, Adventure, Comedy, Action",1000003 An aging Chinese immigrant is swept up...,"[Escapist, Feel-good]"
3,1000004,Fight Club,1999.0,Mischief. Mayhem. Soap.,A ticking-time-bomb insomniac and a slippery s...,139.0,4.27,"Intense violence and sexual transgression, Hum...",Drama,1000004 A ticking-time-bomb insomniac and a sl...,[Emotional]
4,1000005,La La Land,2016.0,Here's to the fools who dream.,"Mia, an aspiring actress, serves lattes to mov...",129.0,4.09,"Song and dance, Humanity and the world around ...","Drama, Comedy, Music, Romance","1000005 Mia, an aspiring actress, serves latte...","[Artistic, Emotional, Feel-good]"
...,...,...,...,...,...,...,...,...,...,...,...
24414,1664306,Fixerr,2019.0,NaN,ATS Officer Jayveer Malik’s stands suspended a...,324.0,NaN,"Thrillers and murder mysteries, High speed and...",Drama,1664306 ATS Officer Jayveer Malik’s stands sus...,[Emotional]
24415,1693420,"LSD - Love, Scandal and Doctors",2021.0,NaN,"Love, Scandal and Doctors is a story of 5 medi...",300.0,NaN,"Thrillers and murder mysteries, High speed and...","Drama, Mystery","1693420 Love, Scandal and Doctors is a story o...","[Emotional, Suspenseful]"
24416,1702871,Olessia,2019.0,NaN,Film with a soundtrack featuring G. Venugopal ...,NaN,NaN,"Song and dance, Dazzling vocal performances an...",Drama,1702871 Film with a soundtrack featuring G. Ve...,[Emotional]
24417,1762425,Pretty Things,2005.0,NaN,A look into the world of 20th century burlesqu...,90.0,NaN,"Song and dance, Erotic relationships and desire",Documentary,1762425 A look into the world of 20th century ...,[Educational]


### At this point of coding I am thinking categorizing based off THEME instead of GENRE might be the better move. But for starting purposes, I am keeping it with genres

In [19]:
# checking to see if the GPU is available, in my case it is not
import torch
print(torch.cuda.is_available())

False
False


In [20]:
from transformers import pipeline


pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=-1)


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

C:\Users\Owner\PycharmProjects\movie-recommender\.venv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Owner\.cache\huggingface\hub\models--facebook--bart-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\Owner\PycharmProjects\movie-recommender\.venv\Lib\site-packages\huggingface_hub\

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Device set to use cpu


In [65]:
genre_categories = [
    'Artistic',
    'Classic',
    'Educational',
    'Emotional',
    'Escapist',
    'Feel-good',
    'Suspenseful'
]
# print(genre_categories)
# sequence = movies.loc[movies["simple_genres"] == "Drama", "description"].reset_index(drop=True)[0]
drama_movies = movies[movies["simple_genres"].apply(lambda x: "Emotional" in x)]
sequence = drama_movies["description"].reset_index(drop=True)[0]
sequence
# movies

"All unemployed, Ki-taek's family takes peculiar interest in the wealthy and glamorous Parks for their livelihood until they get entangled in an unexpected incident."

### Using argmax, we will see what the model yields the most likely category for this specific movie

In [67]:
import numpy as np


# results = pipe(sequence, genre_categories)
# print("Results before sorting:")
# print("Labels:", results['labels'])
# print("Scores:", results['scores'])
# print(results)
# Sort the results based on scores in descending order
# sorted_indices = np.argsort(results['scores'])[::-1]
#
# # # Get top 3 labels and scores
# top_labels = [results['labels'][i] for i in sorted_indices[:3]]
# top_scores = [results['scores'][i] for i in sorted_indices[:3]]
# # print(top_scores)
# # Print top 3 labels and scores
# for label, score in zip(top_labels, top_scores):
#     print(f"{label}: {score:.4f}")

max_index = np.argmax(pipe(sequence, genre_categories)["scores"])
max_label = pipe(sequence, genre_categories)["labels"][max_index]
max_label

'Suspenseful'

'Suspenseful'